# intermittent time series

In [14]:
import numpy as np
import pandas as pd
import seaborn as sns


import matplotlib
from matplotlib import pyplot as plt
from textwrap import wrap


pd.options.display.float_format = '{:,.2f}'.format
pd.options.display.max_columns = None
pd.options.display.max_rows = None
pd.set_option('display.max_colwidth', None)
pd.options.mode.chained_assignment = None  # default='warn'

import pickle
import time

from itertools import product

#import googletrans
#from googletrans import Translator
#to avoid instantiate too many translator, we created one instance for the whole notebook
#translator = Translator()


from prophet import Prophet
import itertools
from prophet.diagnostics import cross_validation
from prophet.diagnostics import performance_metrics

import holidays


import hts.functions
import collections
from hts import HTSRegressor
from hts.hierarchy import HierarchyTree


from sktime.transformations.hierarchical.reconcile import Reconciler
from sktime.transformations.hierarchical.aggregate import Aggregator
from sktime.forecasting.exp_smoothing import ExponentialSmoothing




In [120]:
#!pip install holidays

In [121]:
#!pip install prophet

In [122]:
#https://github.com/matthiasanderer/m5-accuracy-competition/blob/main/m5-simple-fe-evaluation.ipynb

## Memory Reducer
# :df pandas dataframe to reduce size             # type: pd.DataFrame()
# :verbose                                        # type: bool
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                       df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

# import data from pickle file

In [123]:
#read pickle_dict, which store everything
pickle_dict1 = pickle.load(open('../temp/data1.pkl', 'rb'))
df_all = pickle_dict1['df_all']   #concat df_train_m and df_test
df_train = pickle_dict1['df_train']
df_test = pickle_dict1['df_test']
df_items = pickle_dict1['df_items']
df_shops = pickle_dict1['df_shops']
df_cat = pickle_dict1['df_cat']
new_arrival_item_list = pickle_dict1['new_arrival_item_list']

### create plain grid and merge df_all --> df_grid2

In [124]:
#https://www.kaggle.com/code/szhou42/predict-future-sales-top-11-solution/notebook
# For every month we create a grid from all shops/items combinations from that month
grid = []
for block_num in df_all['date_block_num'].unique():
    cur_shops = df_all[df_all['date_block_num']==block_num]['shop_id'].unique()
    cur_items = df_all[df_all['date_block_num']==block_num]['item_id'].unique()
    grid.append(np.array(list(product(*[cur_shops, cur_items, [block_num]])),dtype='int32'))
index_cols = ['shop_id', 'item_id', 'date_block_num']

#plain grid dataframe
df_grid1 = pd.DataFrame(np.vstack(grid), columns = index_cols,dtype=np.int32)


In [125]:
df_grid1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2946018 entries, 0 to 2946017
Data columns (total 3 columns):
 #   Column          Dtype
---  ------          -----
 0   shop_id         int32
 1   item_id         int32
 2   date_block_num  int32
dtypes: int32(3)
memory usage: 33.7 MB


In [126]:

#merge df_all to df_grid1
df_grid2 = pd.merge(df_grid1,df_all,how='left',on=index_cols)

shop_list = df_grid2['shop_id'].unique()
print("how many shops: ", len(shop_list))

item_list = df_grid2['item_id'].unique()
print("how many items: ", len(item_list))

cat_list = df_grid2['item_category_id'].unique()
print("how many categories: ", len(cat_list))

df_grid2.info()

how many shops:  42
how many items:  5100
how many categories:  63
<class 'pandas.core.frame.DataFrame'>
Int64Index: 2946018 entries, 0 to 2946017
Data columns (total 7 columns):
 #   Column            Dtype  
---  ------            -----  
 0   shop_id           int32  
 1   item_id           int32  
 2   date_block_num    int32  
 3   item_category_id  float64
 4   item_price        float64
 5   item_cnt_month    float64
 6   is_new_arrival    float64
dtypes: float64(4), int32(3)
memory usage: 146.1 MB


In [127]:
df_grid2.isnull().sum()

shop_id                   0
item_id                   0
date_block_num            0
item_category_id    2131660
item_price          2131660
item_cnt_month      2131660
is_new_arrival      2131660
dtype: int64

### df_grid2: handle null value, drop columns --> df_grid3

In [128]:
#handle null value
df_grid2['item_cnt_month'] = df_grid2['item_cnt_month'].fillna(0).astype(int)
df_grid2['item_price'] = df_grid2['item_price'].fillna(0)
df_grid2['is_new_arrival'] = np.where(df_grid2['item_id'].isin(new_arrival_item_list), 1, 0) 

#drop this column
df_grid3=df_grid2.drop(columns=['item_category_id'])

#create new column
df_grid3["month"] = ((df_grid3["date_block_num"]+1)%12).astype(int)
df_grid3['year'] = ((df_grid3["date_block_num"]+1)//12 + 2013).astype(int)
df_grid3['year'] = np.where(df_grid3['month']==0, df_grid3['year']-1,df_grid3['year'] )
df_grid3['month'] = np.where(df_grid3['month']==0, 12,df_grid3['month'] )

#create 'year_month'
df_grid3['year_month'] = df_grid3.apply(lambda x:f"{int(x.year)}-{int(x.month)}",axis=1).apply(pd.to_datetime)
df_grid3['year_month'].unique()


array(['2013-01-01T00:00:00.000000000', '2013-02-01T00:00:00.000000000',
       '2013-03-01T00:00:00.000000000', '2013-04-01T00:00:00.000000000',
       '2013-05-01T00:00:00.000000000', '2013-06-01T00:00:00.000000000',
       '2013-07-01T00:00:00.000000000', '2013-08-01T00:00:00.000000000',
       '2013-09-01T00:00:00.000000000', '2013-10-01T00:00:00.000000000',
       '2013-11-01T00:00:00.000000000', '2013-12-01T00:00:00.000000000',
       '2014-01-01T00:00:00.000000000', '2014-02-01T00:00:00.000000000',
       '2014-03-01T00:00:00.000000000', '2014-04-01T00:00:00.000000000',
       '2014-05-01T00:00:00.000000000', '2014-06-01T00:00:00.000000000',
       '2014-07-01T00:00:00.000000000', '2014-08-01T00:00:00.000000000',
       '2014-09-01T00:00:00.000000000', '2014-10-01T00:00:00.000000000',
       '2014-11-01T00:00:00.000000000', '2014-12-01T00:00:00.000000000',
       '2015-01-01T00:00:00.000000000', '2015-02-01T00:00:00.000000000',
       '2015-03-01T00:00:00.000000000', '2015-04-01

In [129]:
df_grid3.isnull().sum()

shop_id           0
item_id           0
date_block_num    0
item_price        0
item_cnt_month    0
is_new_arrival    0
month             0
year              0
year_month        0
dtype: int64

In [130]:
df_grid3.sample(10)

,shop_id,item_id,date_block_num,item_price,item_cnt_month,is_new_arrival,month,year,year_month
2065645,25,19885,29,0.00,0,0,6,2015,2015-06-01
2569379,2,1100,33,0.00,0,0,10,2015,2015-10-01
1009385,35,10300,19,499.00,1,0,8,2014,2014-08-01
580290,16,521,13,0.00,0,0,2,2014,2014-02-01
2492102,34,13573,32,0.00,0,0,9,2015,2015-09-01
1649342,59,4424,25,0.00,0,0,2,2015,2015-02-01
291291,45,18411,7,0.00,0,0,8,2013,2013-08-01
40279,16,15438,1,0.00,0,0,2,2013,2013-02-01
1587256,26,2424,25,0.00,0,0,2,2015,2015-02-01
1351009,16,6661,23,0.00,0,0,12,2014,2014-12-01


### df_grid3: add 'item_category_id'  =>df_grid4

In [131]:
df_grid4 = pd.merge(df_grid3,df_items[['item_id','item_category_id']],how='left',on='item_id')

In [132]:
df_grid4.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2946018 entries, 0 to 2946017
Data columns (total 10 columns):
 #   Column            Dtype         
---  ------            -----         
 0   shop_id           int32         
 1   item_id           int32         
 2   date_block_num    int32         
 3   item_price        float64       
 4   item_cnt_month    int32         
 5   is_new_arrival    int32         
 6   month             int32         
 7   year              int32         
 8   year_month        datetime64[ns]
 9   item_category_id  int64         
dtypes: datetime64[ns](1), float64(1), int32(7), int64(1)
memory usage: 168.6 MB


In [133]:
df_grid4.isnull().sum()

shop_id             0
item_id             0
date_block_num      0
item_price          0
item_cnt_month      0
is_new_arrival      0
month               0
year                0
year_month          0
item_category_id    0
dtype: int64

### df_grid4 =>df_basegrid

In [134]:
base_column_list = ['date_block_num','item_category_id','shop_id','item_id','item_cnt_month', 'year_month' 
                    #'item_price','is_new_arrival', 'year', 'month'
                   ]

df_basegrid = df_grid4
df_basegrid.head()

,shop_id,item_id,date_block_num,item_price,item_cnt_month,is_new_arrival,month,year,year_month,item_category_id
0,2,5572,0,"1,532.86",9,0,1,2013,2013-01-01,2
1,2,5643,0,"2,387.61",0,0,1,2013,2013-01-01,2
2,2,5583,0,594.15,2,0,1,2013,2013-01-01,5
3,2,7893,0,"1,990.00",3,0,1,2013,2013-01-01,6
4,2,7894,0,"1,490.00",1,0,1,2013,2013-01-01,6


In [90]:
df_basegrid.shape

(2946018, 10)

In [135]:
df_basegrid=reduce_mem_usage(df_basegrid, verbose=True)

Mem. usage decreased to 81.48 Mb (51.7% reduction)


In [136]:
df_basegrid.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2946018 entries, 0 to 2946017
Data columns (total 10 columns):
 #   Column            Dtype         
---  ------            -----         
 0   shop_id           int8          
 1   item_id           int16         
 2   date_block_num    int8          
 3   item_price        float16       
 4   item_cnt_month    int16         
 5   is_new_arrival    int8          
 6   month             int8          
 7   year              int16         
 8   year_month        datetime64[ns]
 9   item_category_id  int8          
dtypes: datetime64[ns](1), float16(1), int16(3), int8(5)
memory usage: 81.5 MB


In [137]:
df_basegrid.sample(5)

,shop_id,item_id,date_block_num,item_price,item_cnt_month,is_new_arrival,month,year,year_month,item_category_id
2179009,18,21980,30,0.00,0,0,7,2015,2015-07-01,57
194611,25,849,5,0.00,0,0,6,2013,2013-06-01,78
2555998,57,12764,32,0.00,0,0,9,2015,2015-09-01,37
671185,50,12542,14,0.00,0,0,3,2014,2014-03-01,55
2783098,16,2698,34,"2,500.00",0,0,11,2015,2015-11-01,20


# pickle

In [138]:
#save neccessary object to pickle_dict, which will be pickled in other notebook for feature engineering and modeling
#
pickle_dict2 = dict()
pickle_dict2['df_basegrid']=df_basegrid   #concat df_train_m and df_test
#pickle_dict2['df_train']=df_train
#pickle_dict2['df_test']=df_test
#pickle_dict2['df_items']=df_items
#pickle_dict2['df_shops']=df_shops
#pickle_dict2['df_cat']=df_cat
pickle.dump(pickle_dict2, open('../temp/data2.pkl', 'wb'))

